In [72]:
# pip install watchdog

In [73]:
import sys
import time
import logging

from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import FileSystemEventHandler

from IPython.display import display
from IPython.display import clear_output
from ipywidgets import IntProgress

import json

import pandas as pd

In [74]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 20;

<IPython.core.display.Javascript object>

In [93]:
def observe(path = "./"):

    files_to_be_created = 100

    pb = IntProgress(description='Iter 0/{}'.format(files_to_be_created), min=0, max=files_to_be_created)
    display(pb)

    class EventHandler(FileSystemEventHandler):
        def on_created(self, event):
            pb.value += 1
            pb.description = 'Iter {}/{}'.format(pb.value, files_to_be_created)
            clear_output(wait=True)
            display(pb)
            on_file_created(event.src_path)

    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    event_handler = EventHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=True)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

In [101]:
df = pd.DataFrame(columns=['name','neg', 'neu', 'pos', 'compound'])

def on_file_created(filepath):
    global df
    
    if(not filepath.endswith(".json")):
        print(df)
        return
    
    with open(filepath) as json_file:
        json_data = json.load(json_file)
#         print(json_data)
        df = df.append({
            'name': json_data['name'], 
            'neg': json_data['emotions']['neg'],
            'neu': json_data['emotions']['neu'],
            'pos': json_data['emotions']['pos'],
            'compound': json_data['emotions']['compound']
        }, ignore_index=True)
    
        df = df.sort_values(by=['compound'], ascending=False).reset_index(drop=True)
        
        mean_text = 'Mean: {}'.format(round(df["compound"].mean(),3))
        display(mean_text)
        display(df)
        
observe(path = '../responses/')

IntProgress(value=100, description='Iter 100/100')

'Mean: 0.462'

,name,neg,neu,pos,compound
0,Page #40,0.228,0.175,0.933,0.992
1,Page #17,0.541,0.784,0.717,0.976
2,Page #27,0.622,0.701,0.393,0.946
3,Page #7,0.533,0.148,0.981,0.941
4,Page #94,0.884,0.532,0.654,0.926
5,Page #63,0.932,0.271,0.425,0.910
6,Page #59,0.306,0.391,0.663,0.904
7,Page #70,0.064,0.587,0.487,0.893
8,Page #98,0.992,0.428,0.405,0.885
9,Page #95,0.366,0.823,0.930,0.883
